## Build Agent

In this tutorial, we'll explore how to create flexible and efficient AI agents using Large Language Models (LLMs). 
We'll cover everything from basic API interactions to building specialized agents for different use cases.

### 1. Setting Up the Environment

In [1]:
from rich import print as rprint
from rich.markdown import Markdown
from openai import OpenAI
import dotenv
dotenv.load_dotenv()

True

### Connecting to OpenAI

There are two methods to connect to the OpenAI API:

#### Method 1: Direct API Key
```python
client = OpenAI(
    api_key="sk-...",
    base_url="https://api.openai.com/v1",
)  # Replace with your API key
```

#### Method 2: Using Environment Variables (Recommended)
Create a `.env` file in your project directory with:
```
OPENAI_API_KEY="<YOUR_API_KEY>"
OPENAI_BASE_URL=<YOUR_BASE_URL>
```

In [2]:
client = OpenAI()

### 2. Understanding Key Parameters

To effectively use the API, it's important to understand some key parameters:

- **model**: The specific LLM you want to use (we'll use gpt-4o-mini)
- **temperature**: Controls response randomness (0.0 to 1.0)
  - Higher values (closer to 1.0) = more creative responses
  - Lower values (closer to 0.0) = more consistent responses
- **system_prompt**: Instructions that define how the LLM should behave

In [3]:
# Define default parameters
model = "gpt-4o-mini"
temperature = 0.0

### 3. Basic API Interaction

In [4]:
# Example of a basic system prompt
system_prompt = """
You are a helpful AI assistant that provides accurate and concise responses.
You have expertise in building virtual assistants and AI systems.
Please provide clear explanations and relevant examples when appropriate.
"""

# Function to make API calls
def create_content(
    query: str, 
    client: OpenAI, 
    system_prompt: str, 
    model: str, 
    temperature: float
) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

create_content(
    query="What is the capital of France?",
    client=client,
    system_prompt=system_prompt,
    model=model,
    temperature=temperature)

'The capital of France is Paris.'

### 4. Building a Flexible Agent Class

Now, let's build a flexible Agent class that can be customized for different purposes:

**Constructor**

First of all, create a class called Agent with the following parameters in the `__init__` method:
- name (default: "Agent")
- role (default: "Personal Assistant")
- instructions (default: "Help users with any question")
- model (default: "gpt-4o-mini")
- temperature (default: 0.0)

It's recommended having the client being accessible inside your agent.


**Invoke**

Most agentic framworks out there provide an `invoke()` method. For compatibilities reason, we'll do the same. This method should:
- take a message as input;
- send the message to the LLM's API using the specified model and temperature;
- format the API request with the system role and user input;
- return the LLM’s response.

Notice that your system prompt must consider role and instructions, otherwise it will not behave as you'd like.



In [5]:
class Agent:
    def __init__(
        self,
        name: str = "Agent",
        role: str = "Personal Assistant",
        instructions: str = "Help users with any question",
        model: str = "gpt-4o-mini",
        temperature: float = 0.0,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.agent = OpenAI()

    def invoke(self, message: str) -> str:
        system_prompt = f"You are a {self.role} that {self.instructions}"
        user_question = message
        response = self.agent.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_question},
            ],
            temperature=self.temperature,
        )
        return response.choices[0].message.content

### 5. Example Use Cases

#### 5.1 Marketing Analyst Example

In [6]:
marketing_system_prompt = """
You are a Marketing Analyst at a digital marketing agency. You help create engaging content 
for clients across different industries and channels.

You should:
- Understand the client's industry, target audience and goals
- Create content tailored for specific channels (email, social media, etc.)
- Use appropriate tone and style for each platform
- Include relevant calls-to-action and engagement hooks
- Follow best practices for each content type
"""

marketing_agent = Agent(
    role="Marketing Analyst",
    instructions=marketing_system_prompt,
    temperature=0.3
)

analyst_query = "Create an instagram post for clients in the automotive industry"
marketing_response = marketing_agent.invoke(analyst_query)
rprint(Markdown(marketing_response))

Instagram Post for Automotive Industry Client                                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Image/Video: A high-quality image of a sleek, new car model parked in a scenic location, or a short video          
showcasing the car in action (e.g., driving through a winding road).                                               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Caption:                                                                                                           

🚗✨ Unleash the Road Ahead! ✨🚗                                                                                  

Get ready to experience the thrill of driving like never before! Our latest model combines cutting-edge technology,
stunning design, and unmatched performance. Whether you're navigating city streets or hitting the open road, this  
car is built for adventure.                                                                                        

🔑 Key Features:                                                                                                   

 • Advanced safety systems for peace of mind                                                                       
 • Eco-friendly engine options for a sustainable drive                                                             
 • Luxurious interior for ultimate comfort                                                                         

👉 Ready to take the wheel? Book a test drive today and feel the difference! Link in bio.                          

#Automotive #CarLovers #TestDrive #LuxuryCars #Innovation #DriveYourDreams                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Engagement Hook: "What's your dream road trip destination? Share with us in the comments! 🌍🚙"                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Best Practices:                                                                                                    

 • Use high-quality visuals to capture attention.                                                                  
 • Keep the caption concise yet informative.                                                                       
 • Include relevant hashtags to increase reach.                                                                    
 • Use a clear call-to-action to drive engagement and conversions.

#### 5.2 Travel Assistant Example

In [7]:
travel_agent = Agent(
    role="Travel Agent", 
    instructions="Help users plan their vacations"
)
travel_response = travel_agent.invoke("Where should I go for vacation in December?")
rprint(Markdown(travel_response))

December is a wonderful time for a vacation, as it offers a variety of destinations depending on your preferences  
for weather, activities, and experiences. Here are some options to consider:                                       

 1 Tropical Destinations:                                                                                          
    • Hawaii: Enjoy beautiful beaches, hiking, and vibrant culture.                                                
    • Caribbean Islands (like Jamaica, Barbados, or the Bahamas): Perfect for sun, sand, and relaxation.           
    • Mexico (Cancun or Playa del Carmen): Great for beach lovers and those looking for vibrant nightlife.         
 2 Winter Wonderland:                                                                                              
    • Aspen or Vail, Colorado: Ideal for skiing, snowboarding, and cozy mountain lodges.                           
    • Whistler, Canada: A fantastic destination for winter sports and stunning scenery.                            
    • Lapland, Finland: Experience a magical winter with opportunities to see the Northern Lights and visit Santa  
      Claus.                                                                                                       
 3 Cultural Experiences:                                                                                           
    • New York City, USA: Enjoy the holiday lights, ice skating in Central Park, and Broadway shows.               
    • Vienna, Austria: Famous for its Christmas markets, classical music, and beautiful architecture.              
    • Tokyo, Japan: Experience unique holiday lights, shopping, and delicious cuisine.                             
 4 Adventure and Nature:                                                                                           
    • Costa Rica: Great for eco-tourism, with opportunities for hiking, wildlife watching, and beach time.         
    • New Zealand: Enjoy summer activities like hiking, bungee jumping, and exploring stunning landscapes.         
 5 Relaxation and Wellness:                                                                                        
    • Bali, Indonesia: Perfect for a wellness retreat, with beautiful resorts and serene beaches.                  
    • Sedona, Arizona: Known for its stunning red rock formations and wellness spas.                               

Consider what type of experience you’re looking for—whether it’s relaxation, adventure, culture, or winter         
sports—and choose a destination that aligns with your interests!

#### 5.3 Creative Storyteller Example

In [8]:
storyteller_agent = Agent(
    role="Creative Storyteller", 
    instructions="Help users create imaginative stories", 
    temperature=0.7
)
story_response = storyteller_agent.invoke("Tell me a story about a dragon and a wizard")
rprint(Markdown(story_response))

Once upon a time, in a realm where mountains kissed the skies and forests whispered ancient secrets, there existed 
a magnificent dragon named Zephyra. With scales that shimmered like emeralds and wings that spanned across the     
horizon, Zephyra was the guardian of the Misty Peaks, a sanctuary filled with magic and wonder.                    

In a quaint village at the base of these mountains lived a wise and kind-hearted wizard named Eldrin. He was known 
far and wide for his gentle nature and his ability to commune with the elements. Eldrin had a long, silvery beard  
that flowed like a waterfall, and his eyes sparkled with wisdom gathered over centuries.                           

For years, the villagers had lived in harmony with the dragon, who occasionally soared above their homes, casting  
shadow and awe. They revered Zephyra as a protector, believing that her fiery breath kept the dark forces at bay.  
However, a recent drought had swept through the land, leaving the rivers dry and the fields barren. The villagers  
grew anxious, whispering tales of misfortune and calamity.                                                         

One evening, Eldrin decided to seek the counsel of Zephyra, believing that the dragon might hold the key to        
restoring balance to the land. He climbed the winding paths up the Misty Peaks, his heart steady with purpose. When
he reached the dragon's lair, he found Zephyra perched atop a colossal rock, her eyes glimmering like stars.       

“Great Zephyra,” Eldrin called out, his voice echoing through the cavernous space. “The land suffers from drought, 
and the people fear the worst. We seek your wisdom to bring back the rains.”                                       

The dragon turned her fierce gaze towards him, her eyes softening with understanding. “Eldrin, wise wizard, the    
drought is a consequence of the imbalance in the magic that flows through our world. My fire, once a force of      
protection, has begun to wane, and the water spirits grow restless.”                                               

Eldrin pondered her words. “Then we must restore this balance. How can we rekindle your fire and soothe the water  
spirits?”                                                                                                          

Zephyra unfurled her wings and revealed a shimmering amulet hanging from her neck. “This amulet contains the       
essence of both fire and water. It is said that only through a united heart can its power awaken. We must embark on
a quest to the Crystal Caverns where the water spirits dwell. There, we can forge a new bond between fire and      
water.”                                                                                                            

And so, the dragon and the wizard set forth together, their destinies intertwined. They traversed treacherous      
paths, climbed steep cliffs, and crossed raging rivers, facing challenges that tested their resolve. Along the way,
they encountered mystical creatures: a mischievous sprite who played tricks, a wise old owl who shared riddles, and
a band of playful foxes who offered guidance.                                                                      

Finally, they reached the Crystal Caverns, a breathtaking underground world filled with glistening stalactites and 
pools of shimmering water. The air hummed with magic as the water spirits emerged, their forms fluid and ethereal. 

“Great spirits of the water,” Zephyra spoke, her voice resonating with authority. “We come to seek a bond that will
restore balance to our realm. Together, we can ignite a new flame and summon the rains.”                           

The water spirits swirled around them, intrigued by the dragon and wizard's unity. Eldrin stepped forward, holding 
the amulet high. “Let us unite our powers. Fire and water can create life, harmony, and balance.”                  

In that moment, as Zephyra breathed forth he

### 6. Experimentation Ideas

Now that you understand how everything works, here are some ways to experiment:

1. **Modify Agent Parameters**
   - Try different roles and instructions
   - Adjust the temperature to see how it affects creativity
   - Test different models if available

2. **Create Specialized Agents**
   - Design agents for specific industries
   - Create agents with multiple specialties
   - Experiment with different system prompts

3. **Test Different Use Cases**
   - Try complex queries
   - Test the agents with real-world scenarios
   - Combine different agent types for complex tasks

4. **Explore API Features**
   - Inspect the full response object
   - Try different API parameters
   - Implement error handling

Remember: The key to building effective agents lies in crafting clear instructions 
and understanding how different parameters affect the output.